In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
##Thanks to jamartinh on GH for demystifying how to implement on keras.
class Callback(tf.keras.callbacks.Callback):
    def __init__(self, x_train, y_train, x_val, y_val):
        self.x = x_train
        self.y = y_train
        self.x_val = x_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc_train = roc_auc_score(self.y, y_pred)
        y_pred_val= self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc_train: %s - roc-auc_val: %s' % (str(round(roc_train, 4)), str(round(roc_val, 4))),
              end=100 * ' ' + '\n')
        return
    
encoder = LabelEncoder()
scaler = MinMaxScaler()

x = pd.read_csv("../input/loan-prediction-based-on-customer-behavior/Training Data.csv")
x = pd.concat([x.loc[x["Risk_Flag"]==0][:30996], x.loc[x["Risk_Flag"]==1]])
y = x.pop("Risk_Flag")
str_x = x.select_dtypes(include=[object])

for i in range(0, len(str_x.columns)):
    x.pop(str_x.columns[i])

x = scaler.fit_transform(x)
x = pd.DataFrame(x)
str_x = str_x.apply(encoder.fit_transform)
str_x = pd.DataFrame(str_x)
str_x.index = x.index
x = pd.concat([x, str_x], axis=1)

x_train, x_val, y_train, y_val = train_test_split(x, y, train_size=0.8, shuffle=True)

In [ ]:
dense_1 = tf.keras.layers.Dense(192, activation='relu', input_dim=12)
dense_2 = tf.keras.layers.Dense(128, activation='relu')
dense_3 = tf.keras.layers.Dense(64, activation='relu')
dense_4 = tf.keras.layers.Dense(32, activation='relu')
output = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.models.Sequential([
    dense_1,
    tf.keras.layers.Dropout(.4),
    dense_2,
    tf.keras.layers.Dropout(.2),
    dense_3,
    dense_4,
    output
])

callback = Callback(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=75, batch_size=512, callbacks=[callback])

In [ ]:
plt.xlabel("Epochs")
plt.ylabel("Validation")

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(["acc", "validation acc"])
plt.show()

plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(["loss", "validation loss"], loc="upper left")
plt.show()